# 符号定义
待优化参数w、损失函数loss、学习率lr、每迭代一个batch，t表示当前batch迭代的总次数
1. 计算t时刻损失函数关于当前参数的梯度${g_t} = {Δloss} $=   ${əloss} \over əw_t$
2. 计算t时刻一阶动量$m_t$和二阶动量$V_t$
    + 一阶动量：与梯度相关的函数
    + 二阶动量：与梯度平方相关的函数
3. 计算t时刻下降梯度：$ŋ_t = lr * m_t / \sqrt{V_t}$(学习率乘以一阶动量除以根号下二阶动量)

4. 计算t+1时刻参数：$W_{t+1} = W_t - ŋ_t = W_t - lr * m_t / \sqrt{V_t}$（当前时刻的参数减去学习率✖）

## SGD 无momentum，常用的梯度下降
$W_{t+1} = $ $W_t - lr * $ $əloss \over əw_t^*$

    w1.assign_sub(lr * grads[0])
    b1.assign_sub(lr * grads[1])

## SGDM 含有momentum的SGD，在SGD基础增加一阶动量
$m_t = β * m_{t-1} + (1 - β) * g_t$

而且上一时刻的一阶动量$m_{t-1}$占大头，因为β经验值接近0.9
二阶动量在SGDM中仍是恒等于1的 $V_t = 1$

把一阶动量和二阶动量带入ŋ的计算公式，

$ŋ_t = lr * m_t / \sqrt{V_t}$ = $lr * m_t$ = $ lr * (β * m_{t-1} + (1-β) * g_t)$

再把ŋ带入参数更新公式就可以用SGDM更新参数了

$W_{t+1} = W_t - ŋ_t$ = $ W_t - lr * (β * m_{t-1} + (1 - β) * g_t)$

用python实现的最重要的一步是把一阶动量和二阶动量计算出来

$m_t = β * m_{t-1} + (1 - β) * g_t$

$g_t$是当前时刻的梯度

    m_w, m_b = 0,0 # 第0时刻的一阶动量
    beta = 0.9
    
    # sgd-momentum
    m_w = beta * m_w + (1 - beta) * grads[0]
    m_b = beta * m_b + (1 - beta) * grads[1]
    w1.assign_sub(lr * m_w)
    b1.assign_sub(lr * m_b)


# Adagrad，在SGD的基础上增加二阶动量
`可以对模型中的每个参数分配自适应学习率了`

adagrad的一阶动量和SGD一样，是当前的梯度
$m_t = g_t$

二阶动量是从开始到现在、梯度平方的累加和
$V_t = \sum^{t}_{t=1} g_t^2$

有了一阶动量和二阶动量，可以带入参数更新参数了
$ŋ_t = lr * m_t / (\sqrt{V_t})$  = $lr * g_t / (\sqrt{\sum g_t^2})$

$W_{t+1} = W_t - ŋ_t = W_t - lr * g_t / (\sqrt{\sum g_t^2})$

adagrad的一阶动量mt就是当前时刻的梯度
$m_t = g_t  V_t = \sum^{t}_{t=1} g_t^2$

所以直接带入参数更新公式
设0时刻 w和b的二阶动量初始值是0

    v_w,v_b = 0,0
    # adagrad
    v_w += tf.square(grads[0])# 梯度平方累计和
    v_b += tf.square(grads[1])
    w1.assign_sub(lr * grads[0] / tf.sqrt(v_w))
    b1_assign_sub(lr * grads[1] / tf.sqrt(v_b))


# RMSProp是在SGD基础上增加二阶动量

二阶动量v使用煮熟滑动平均值计算，表征的是过去一段时间的平均值

同样，求出一阶动量和二阶动量后

$m_t = g_t$
$V_t = β * V_{t-1} + (1 - β) * g^2_t$

代入参数更新公式
$ŋ_t = lr * m_t / (\sqrt{V_t})$  = $ lr * g_t / (\sqrt{β * V_{t-1} + (1 - β) * g_t^2} )$

$W_{t+1} = W_t - ŋ_t$ = $w_t - lr * g_t /(\sqrt{β * V_{t-1} + (1 - β) * g_t^2} ) $

实现参数自更新

一阶动量$m_t$是梯度

$M_t = g_t V_t = β * V_{t-1} + (1 - β) * g^2_t$
    
    v_w,v_b = 0,0
    beta = 0.9
    v_w = beta * v_w + (1 - beta) * tf.square(grads[0])
    v_b = beta * v_b + (1 - beta) * tf.square(grads[1])
    w1.assign_sub(lr * grads[0] / tf.sqrt(v_w)
    b1.assign_sub(lr * grads[1] / tf.sqrt(v_b)

# adam 

同时结合SGDM一阶动量和RMSProp二阶动量，并在此基础上增加了`两个修正项`
$m_t = β_1 * m_{t-1} + (1 - β_1) * g_t$

修正一阶动量的偏差：$m_t $=$ m_t \over {1 - β_1^t}$

$V_t = β_2 * V_{step-1} + (1 - β_2) * g_t^2$

修正二阶动量的偏差：$V_t = V_t \over (1 - β_2^t)$

$ŋ_t = lr * m_t / \sqrt{V_t }$ 

$W_{t+1} = W_t - ŋ_t$

    m_w，m_b = 0,0
    v_w,v_b = 0,0
    beta1, beta2 = 0.9,0.999
    delta_w, delta_b = 0,0
    global_step = 0
    # global_step 是训练开始到当前时刻所经历的总batch数
    
    #adam一阶动量表达式和含momentum的sgd一阶动量表达式是相同的
    m_w = beta1 * m_w + (1 - beta1) * grads[0]
    m_b = beta1 * m_b + (1 - beta1) * grads[1]    
    # 二阶动量表达式和RMSProp的二阶动量表达式是一样的
    v_w = beta2 * v_w + (1 - beta2) * tf.square(grads[0])
    v_b = beta2 * v_b + (1 - beta2) * tf.square(grads[1])
    
    m_w_correction = m_w / (1 - tf.pow(beta1,int(global_step)))
    m_b_correction = m_b / (1 - tf.pow(beta1,int(global_step)))
    v_w_correction = v_w / (1 - tf.pow(beta2,int(global_step)))
    v_b_correction = v_b / (1 - tf.pow(beta2,int(global_step)))
    
    # 把修正项带入参数更新公式
    w1.assign_sub(lr * m_w_correction / tf.sqrt(v_w_correction))
    b1.assign_sub(lr * m_b_correction / tf.sqrt(v_b_correction))